In [1]:
from splinter import Browser
from selenium.common.exceptions import InvalidElementStateException
from selenium.common.exceptions import ElementNotInteractableException
from bs4 import BeautifulSoup
import pandas as pd
import html5lib
import shutil
import time
import urllib3
import requests

ModuleNotFoundError: No module named 'requests'

In [2]:
# Set the zip codes
codes = [i for i in range(27006, 28909)]
# codes.append('27606')   # just hard code one in
code = codes[0]    # set the variable for zip_and_click
len(codes)

1903

In [3]:
# inititial website to visit
url = 'https://weather.com'

In [4]:
def chrome():
    '''
    Finds the chromedriver in the system and creates a Chrome browser
    '''
    executable_path = {'executable_path': shutil.which('chromedriver')}
    browser = Browser('chrome', **executable_path)
    return(browser)

In [5]:
def zip_and_click(code):
    '''
    Enter zip codes into the search bar on weather.com and click the first result.
    No return, just leaves browser at the first data page.
    '''
    import time
    
    filled = False
    clicked = False
    
    inputs = browser.find_by_tag('input') # get the search box reference
    search_box = inputs[0]
    while not filled:
        try:
            search_box.fill(code)
            filled = True
        except InvalidElementStateException:
            time.sleep(1)
    while not clicked:
        try:
            browser.click_link_by_partial_href('/weather/today/l')
            print('no exception this time')
            clicked = True
        except ElementNotInteractableException:
            print('exception found....waiting...')
            time.sleep(1)
    return(browser)     

In [6]:
def scrape_now(browser):
    ''' Get observed weather data from the current weather page '''
    divs = browser.find_by_tag('div').first
    nowcard = divs.find_by_tag('section.today_nowcard-container').value
    nowlist = nowcard.split('\n')
    return(nowlist)

In [7]:
def wind_dict(wind_str):
    ''' handle the cases where the wind value is simply "calm" rather than direction and speed '''
    if wind_str == 'Calm':
        return {
            'speed': 0,
            'direction': None
        }
    else:
        return {
            'speed': wind_str.split()[1],
            'direction': wind_str.split()[0]
        }

In [8]:
def scrape_next36(browser):
    ''' Get the "at a glance" data from the next 36 hours '''

    next_36 = {}
    
    # Pulling data from the "next 36 hours" overview displays. There are 5 sections
    # that are pulled from. Each section gets a different label.
    hour_code = {'0':'now',
                '1':'at_start',
                '2':'at_12',
                '3':'at_24',
                '4':'at_36'
                }
    for i in range(5):
        print(f'looking for {hour_code[str(i)]} snapshot')
        browser.find_by_tag(f'div#daypart-{i}').click()
        snapshot = browser.find_by_tag(f'div#daypart-{i}')
        snap_values = snapshot.value
        details = browser.find_by_tag('span.wx-detail-value')
        detaillist = [details[j].value for j in range(4)]
        print(f'completed section {i} snapshot')

        #store the data
        next_36.update({hour_code[str(i)]: {'hour': 'CALCULATED HOUR OF PREDICTION',
                                            'condition': snap_values.split('\n')[1],
                                            'high/low': snap_values.split('\n')[2],
                                            'temp_f': snap_values.split('\n')[3],
                                            'chance_precip': snap_values.split('\n')[4],
                                            'description': browser.find_by_id(f'dp{i}-details-narrative').first.value,
                                            'wind': wind_dict(detaillist[0]),
#                                             {'speed_mph': detaillist[0].split(' ')[1],
#                                                            'direction': detaillist[0].split(' ')[0]
#                                                    },
                                            'humidity': detaillist[1],
                                            'uv_index': detaillist[2],
                                            'sun': {'rise': detaillist[3],
                                                    'set': detaillist[3]
                                                   }
                                              }
                      })
    return(next_36)

In [9]:
def goto_hourly(browser):
    ''' Take the browser to the hourly numbers '''
    browser.find_by_text('Hourly').click()
    time.sleep(1)
    n=0
    while browser.is_text_present('Next 8 Hours', wait_time=.5):
        n+=1
        print(n)
        next8 = browser.find_by_text('Next 8 Hours')
        print('got next 8 hour spotted')
        next8.click()
        print('clicked and waiting')
        time.sleep(.5)
        print(f'end n={n} th pass')
    return

In [10]:
browser = chrome()

In [54]:
browser.quit()

In [11]:
browser.visit(url)

In [12]:
zip_and_click(code)

exception found....waiting...
no exception this time


In [13]:
now = scrape_now(browser)

In [14]:
# now

In [15]:
next_36 = scrape_next36(browser)

looking for now snapshot
completed section 0 snapshot
looking for at_start snapshot
completed section 1 snapshot
looking for at_12 snapshot
completed section 2 snapshot
looking for at_24 snapshot
completed section 3 snapshot
looking for at_36 snapshot
completed section 4 snapshot


In [17]:
from pprint import pprint
pprint(next_36['now'])
# next_36

{'chance_precip': '30%',
 'condition': 'AM SHOWERS',
 'description': 'A few showers this morning with overcast skies during the '
                'afternoon hours. High 53F. Winds NE at 5 to 10 mph. Chance of '
                'rain 30%.',
 'high/low': 'HIGH',
 'hour': 'CALCULATED HOUR OF PREDICTION',
 'humidity': '44%',
 'sun': {'rise': '6:58 am5:14 pm', 'set': '6:58 am5:14 pm'},
 'temp_f': '53°',
 'uv_index': '3 of 10',
 'wind': {'direction': 'NE', 'speed': '6'}}


In [64]:
browser.is_text_present('Next 8 Hours', wait_time=.5)
# time.wait(.5)

False

In [38]:
next8 = browser.find_by_text('Next 8 Hours')
next8.click()

NameError: name 'wait' is not defined

In [16]:
goto_hourly(browser)

1
got next 8 hour spotted
clicked and waiting
end n=1 th pass
2
got next 8 hour spotted
clicked and waiting
end n=2 th pass
3
got next 8 hour spotted
clicked and waiting
end n=3 th pass
4
got next 8 hour spotted
clicked and waiting
end n=4 th pass


In [71]:
def scrape_hourly(browser):
    

SyntaxError: unexpected EOF while parsing (<ipython-input-71-74de2fd0a621>, line 2)

In [25]:
import requests
header = {
  "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.97 Safari/537.36",
  "X-Requested-With": "XMLHttpRequest"
}
r = requests.get(url, headers=header)
dfs = pd.read_html(r.text)

ModuleNotFoundError: No module named 'requests'

In [18]:
url = browser.url
print(url)
tables = pd.read_html(url)
tables

https://weather.com/weather/hourbyhour/l/15d7cae6a3bd8dea6586e2ea77ef72be81e7593b073dcd3ed31cf719e686f364


HTTPError: HTTP Error 403: Forbidden

In [20]:
browser.find_by_tag('div#daypart-0').click()
snapshot = browser.find_by_tag('div#daypart-0').value
snaplist = snapshot.split('\n')
details = browser.find_by_tag('span.wx-detail-value')
detailslist = [details[i].value for i in range(4)]
print(snaplist)
detailslist

['TODAY', 'SUNNY', 'HIGH', '51°', '0%']


['ESE 5 mph', '46%', '4 of 10', '6:44 am5:12 pm']

In [49]:
print(snaplist)

['TONIGHT', 'RAIN', 'LOW', '42°', '90%']


In [43]:
details = exec("browser.find_by_tag('span#dp0-details-wind')")
# dp_details = {}
# for i in range(4):
#     dp_details[exec("browser.find_by_tag('span#dp{i}-details-wind'.format(i))").value]
print(details)

None


In [35]:
details = browser.find_by_tag('span.wx-detail-value')
print(details)

In [36]:
for i in range(4):
    print(details[i].value)

NNW 10 mph
86%
0 of 10
6:43 am5:13 pm


In [12]:
# browser.find_by_tag('div.today-daypart-content').first.value

'TODAY\nHIGH\n70°\n10%'

In [26]:
description = browser.find_by_id('dp1-details-narrative').first.value
description

'A clear sky. Scattered frost possible. Low around 30F. Winds light and variable.'

In [40]:
browser.quit()

1

'Today'

got input box


1

In [25]:
hour_code = {'0':'now',
            '1':'at_start',
            '2':'at_12',
            '3':'at_24',
            '4':'at_36'
}
print(hour_code['0'])
for i in range(5):
    print(hour_code[str(i)])
    print(f'looking for {hour_code[str(i)]} snapshot')

now
now
looking for now snapshot
at_start
looking for at_start snapshot
at_12
looking for at_12 snapshot
at_24
looking for at_24 snapshot
at_36
looking for at_36 snapshot


In [ ]:
### SEARCH FOR THE TABLE THAT HOLDS THE SEARCH RESULTS ###